## Lempel - Ziv complexity analysis 


Based on Schartner(2017) code. Calcuates LZs , LZc and Power spectral density based on the definitions from the 2017 paper. Uses MNE python for eeg data operations and minor changes to the original script prrovided by Schartner (2017). Input data uses Epoched EEGLAB .set files. 

Script adapted for the LZ correaltion experiment.



In [2]:
import mne
import LZ_Spectral_edited as fl
import numpy as np


from scipy import signal
from scipy.signal import (butter,lfilter,hilbert,resample)
from matplotlib.pylab import *
import os as os

# Channel selection for LZs and PSpec. OZ selected for best effect due to photic stim
channel = 31

# subject number
subject_number = "1"
#subject_number = ['01','02','03','04','05','06','07','08','10','12','13','14','15','16','17','18','19','21','22','23']
subject_length = len(subject_number)
#Different Conditions 
condition = [11,12,13,21,22,23,31,32,33]
condition_length = len(condition)


LZS_data= np.zeros(len(condition) + 1)
LZC_data= np.zeros(len(condition) + 1)
#pspec_data = np.zeros([len(condition) + 1,5])
pspec_data = np.zeros(4)

array_len = (subject_length * condition_length) + 1


dat_boi = np.zeros((array_len,5)) 


def LZc(X):
 '''
 Compute LZc and use shuffled result as normalization
 '''
 X=Pre2(X)
 SC=str_col(X)
 M=list(SC)
 shuffle(M)
 w=''
 for i in range(len(M)):
  w+=M[i]
 return cpr(SC)/float(cpr(w))

def Pre2(X):
 '''
 Linear-detrend and subtract mean of X, a multidimensional times series (channels x observations)
 '''
 ro,co=shape(X)
 Z=zeros((ro,co))
 for i in range(ro): #maybe divide by std?
  Z[i,:]=signal.detrend((X[i,:]-mean(X[i,:]))/std(X[i,:]), axis=0)
 return Z
 
  


def LZs(x):
 
 '''
 Lempel ziv complexity of single timeseries
 '''
 
 ro,co=np.shape(x)
 x=signal.detrend((x-mean(x))/std(x), axis=0)
 s=''
 r=abs(hilbert(x))
 th=mean(r)
 
 for j in range(co):
  if r[0,j]>th:
   s+='1'
  else:
   s+='0'
 
 M=list(s)
 shuffle(M)
 w=''
 for i in range(len(M)):
  w+=M[i]
 
 return cpr(s)/float(cpr(w))

  
def cpr(string):
 '''
 Lempel-Ziv-Welch compression of binary input string, e.g. string='0010101'. It outputs the size of the dictionary of binary words.
 '''
 d={}
 w = ''
 for c in string:
  wc = w + c
  if wc in d:
   w = wc
  else:
   d[wc]=wc   
   w = c
 return len(d)

def PSpec(X):
 '''
 X: multidimensional time series, ch x obs
 fs: sampling rate in Hz
 '''
 
 
 def find_closest(A, target):
   '''
   helper function
   '''
   #A must be sorted
   idx = A.searchsorted(target)
   idx = np.clip(idx, 1, len(A)-1)
   left = A[idx-1]
   right = A[idx]
   idx -= target - left < right - target
   return idx
 
 
 fs=1024
 
 de=[1,4]# in Hz
 th=[4,8]
 al=[8,13]
 be=[13,30]
# ga=[30,60]
# hga=[60,120]
 
 F=[de,th,al,be]#,ga,hga]
 
 ro,co=shape(X)
 Q=[]
 
 for i in range(ro):
 
  v=X[i]
  co=len(v)  
  N = co # Number of samplepoints  
  T = 1.0 / fs # sample spacing (denominator in Hz)
  y = v
  yf = fft(y)
  xf = np.linspace(0.0, 1.0/(2.0*T), int(N/2))
  yff=2.0/N * np.abs(yf[0:int(N/2)])
  bands=zeros(len(F))
  for i in range(len(F)):
   bands[i]=sum(yff[find_closest(xf, F[i][0]):find_closest(xf, F[i][1])])
  bands=bands/sum(bands)
  Q.append(bands)
 return Q
 

In [ ]:
print(array_len)
print(np.shape(dat_boi))

In [3]:
# Index variable for list and file saving
i=1
#LZC_data[0] = "val"
#LZS_data[0] = "val"
for y in subject_number:    
    for x in condition:
        Var_name = "Pilot_%s_%s.set" % (y,x)
        #filename = "%s_ICA.set" % Var_name
        data_file = mne.io.read_raw_eeglab(Var_name)
        fullset = data_file.get_data()
        specific_chandata = data_file.get_data(picks = channel)
        LZC_temp = fl.LZc(fullset)
        LZS_temp = LZs(data_file.get_data(picks = 31))
        dat_boi [i,1] = y
        dat_boi [i,2] = x
        dat_boi [i,3] = LZC_temp
        dat_boi [i,4] = LZS_temp
        #Pspec_temp = PSpec(data_file.get_data(picks = 31))
        #pspec_data = np.vstack((pspec_data,Pspec_temp))
        #LZC_data[i] = LZC_temp
        #LZS_data[i] = LZS_temp
        i = i+1

Reading 01\001_3_ICA.fdt
Reading 01\001_7_ICA.fdt
Reading 01\001_8_ICA.fdt
Reading 01\001_10_ICA.fdt
Reading 01\001_13_ICA.fdt
Reading 01\001_15_ICA.fdt
Reading 01\001_19_ICA.fdt
Reading 01\001_43_ICA.fdt
Reading 02\002_3_ICA.fdt
Reading 02\002_7_ICA.fdt
Reading 02\002_8_ICA.fdt
Reading 02\002_10_ICA.fdt
Reading 02\002_13_ICA.fdt
Reading 02\002_15_ICA.fdt
Reading 02\002_19_ICA.fdt
Reading 02\002_43_ICA.fdt
Reading 03\003_3_ICA.fdt
Reading 03\003_7_ICA.fdt
Reading 03\003_8_ICA.fdt
Reading 03\003_10_ICA.fdt
Reading 03\003_13_ICA.fdt
Reading 03\003_15_ICA.fdt
Reading 03\003_19_ICA.fdt
Reading 03\003_43_ICA.fdt
Reading 04\004_3_ICA.fdt
Reading 04\004_7_ICA.fdt
Reading 04\004_8_ICA.fdt
Reading 04\004_10_ICA.fdt
Reading 04\004_13_ICA.fdt
Reading 04\004_15_ICA.fdt
Reading 04\004_19_ICA.fdt
Reading 04\004_43_ICA.fdt
Reading 05\005_3_ICA.fdt
Reading 05\005_7_ICA.fdt
Reading 05\005_8_ICA.fdt
Reading 05\005_10_ICA.fdt
Reading 05\005_13_ICA.fdt
Reading 05\005_15_ICA.fdt
Reading 05\005_19_ICA.fdt
R

In [ ]:
#Data test 
print (LZC_data)
print (LZS_data)

#ZZ = PSpec(data_file.get_data(picks = 31))
#LZC_data[0] = 'val'
#LZS_data[0] = 'val'

#print(ZZ)

In [4]:
#np.savetxt('LZSfile.csv',LZS_data , delimiter =', ') 
#np.savetxt('LZCfile.csv',LZC_data , delimiter =', ') 
#np.savetxt('pspecboi.csv',pspec_data , delimiter =', ') 


np.savetxt('LZ data boi.csv', dat_boi, delimiter =', ' )

In [12]:
data_file = mne.io.read_raw_eeglab("21/021_43_ICA.set")

LZC_temp = LZc(data_file)
LZS_temp = LZs(data_file.get_data(picks = 31))


print(LZC_temp)
print(LZS_temp)

Reading 21\021_43_ICA.fdt


ValueError: All picks must be < n_channels (64), got 64

In [ ]:
pspec_boi = np.zeros((array_len,7))